<a href="https://colab.research.google.com/github/amjadraza/pycaret/blob/feature%2Fgcp_azure_np_docs/examples/test_notebook_model_deployment_azure_gcp_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip uninstall pycaret
!pip install git+https://github.com/amjadraza/pycaret.git@feature/gcp_azure_np_docs

  Cloning https://github.com/amjadraza/pycaret.git (to revision feature/gcp_azure_np_docs) to /tmp/pip-req-build-2ast6ys_
  Running command git clone -q https://github.com/amjadraza/pycaret.git /tmp/pip-req-build-2ast6ys_
  Running command git checkout -b feature/gcp_azure_np_docs --track origin/feature/gcp_azure_np_docs
  Switched to a new branch 'feature/gcp_azure_np_docs'
  Branch 'feature/gcp_azure_np_docs' set up to track remote branch 'feature/gcp_azure_np_docs' from 'origin'.
     |████████████████████████████████| 6.8MB 7.2MB/s 
     |████████████████████████████████| 266kB 42.3MB/s 
     |████████████████████████████████| 1.2MB 42.8MB/s 
     |████████████████████████████████| 1.6MB 44.2MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 65.9MB 52kB/s 
     |████████████████████████████████| 266kB 48.3MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 12.4MB 43.1MB/s 
     |█████

In [2]:
from pycaret.datasets import get_data
dataset = get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [3]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (5700, 8)
Unseen Data For Predictions: (300, 8)


In [4]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'Price', session_id=123)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(5700, 8)"
4,Missing Values,False
5,Numeric Features,1
6,Categorical Features,6
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [5]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,762.6653,3673803.8234,1916.7169,0.9701,0.0728,0.0523
1,955.5267,7425168.5522,2724.9162,0.9400,0.0850,0.0588
2,852.9126,5146188.1541,2268.5211,0.9579,0.0834,0.0610
3,699.4826,1730518.0345,1315.4916,0.9811,0.0766,0.0557
4,633.4070,1505126.5084,1226.8360,0.9852,0.0681,0.0523
5,758.1128,2698170.1978,1642.6108,0.9727,0.0784,0.0581
6,846.1888,6214502.5417,2492.8904,0.9395,0.0811,0.0582
7,729.4342,2328849.0908,1526.0567,0.9728,0.0765,0.0560
8,837.5604,2614949.9435,1617.0807,0.9738,0.0824,0.0622
9,715.8950,2427369.3891,1558.0017,0.9758,0.0838,0.0580


In [6]:
tuned_lightgbm = tune_model(lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,823.0662,2453046.6128,1566.2205,0.9801,0.0865,0.0632
1,912.8832,5993852.2732,2448.2345,0.9516,0.0856,0.0623
2,832.1467,3108349.3394,1763.0511,0.9745,0.0862,0.0651
3,691.4328,1452486.1837,1205.1913,0.9842,0.0843,0.0595
4,722.3578,1775038.5927,1332.3057,0.9826,0.0805,0.0599
5,738.3989,1958978.3936,1399.6351,0.9802,0.0772,0.0603
6,843.2942,5019902.1173,2240.5138,0.9511,0.0860,0.0625
7,756.8554,2315238.9726,1521.5909,0.9730,0.0801,0.0592
8,837.2992,2257821.7856,1502.6050,0.9774,0.0835,0.0642
9,740.5726,2154216.0963,1467.7248,0.9785,0.0901,0.0650


In [7]:
predict_model(tuned_lightgbm );

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,707.8043,1.828776e+06,1352.3223,0.9821,0.0776,0.0571


# 12.0 Finalize Model for Deployment

In [8]:
final_lightgbm  = finalize_model(tuned_lightgbm )

In [9]:
print(final_lightgbm)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.4, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.9,
              n_estimators=90, n_jobs=-1, num_leaves=10, objective=None,
              random_state=123, reg_alpha=0.9, reg_lambda=0.2, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [10]:
predict_model(final_lightgbm);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,568.6295,880420.4651,938.3072,0.9914,0.0666,0.05


# 13.0 Deploy Model on Microsoft Azure

This is the code to deploy model on Microsft azure using `pycaret` functionalities.

In [11]:
# ! pip install azure-storage-blob
! pip install awscli



     |████████████████████████████████| 3.3MB 8.3MB/s 
     |████████████████████████████████| 6.5MB 45.2MB/s 
  Found existing installation: botocore 1.17.37
    Uninstalling botocore-1.17.37:
      Successfully uninstalled botocore-1.17.37
  Found existing installation: rsa 4.6
    Uninstalling rsa-4.6:
      Successfully uninstalled rsa-4.6


In [ ]:
# Enter connection string when running in google colab
connect_str = '' #@param {type:"string"}
print(connect_str)

In [13]:
#! export AZURE_STORAGE_CONNECTION_STRING=connect_str

In [14]:
import os
os.environ['AZURE_STORAGE_CONNECTION_STRING']= connect_str

In [ ]:
! echo $AZURE_STORAGE_CONNECTION_STRING

In [ ]:
os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [17]:
authentication = {'container': 'pycaret-reg-1011'}
model_name = 'lightgbm-reg-101'
deploy_model(final_lightgbm, model_name, authentication, platform = 'azure')

In [18]:
authentication = {'container': 'pycaret-reg-1011'}
model_name = 'lightgbm-reg-101'
model_azure = load_model(model_name, 
               platform = 'azure', 
               authentication = authentication,
               verbose=True)

Loading model from Microsoft Azure

	lightgbm-reg-101.pkl
Transformation Pipeline and Model Successfully Loaded


In [19]:

unseen_predictions = predict_model(model_azure, data=data_unseen, verbose=True)

In [20]:
predict_model(model_azure)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,568.6295,880420.4651,938.3072,0.9914,0.0666,0.05


,Carat Weight,Cut_Fair,Cut_Good,Cut_Ideal,Cut_Signature-Ideal,Cut_Very Good,Color_D,Color_E,Color_F,Color_G,Color_H,Color_I,Clarity_FL,Clarity_IF,Clarity_SI1,Clarity_VS1,Clarity_VS2,Clarity_VVS1,Clarity_VVS2,Polish_EX,Polish_G,Polish_ID,Polish_VG,Symmetry_EX,Symmetry_G,Symmetry_ID,Symmetry_VG,Report_AGSL,Report_GIA,Price,Label
0,1.60,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,12942,13087.2057
1,1.15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6110,6159.4249
2,2.37,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,28063,26176.5884
3,2.01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,25948,24200.7832
4,0.91,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4381,4480.3517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706,0.90,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,7268,6947.5704
1707,1.02,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,6622,6908.6135
1708,1.54,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,12120,12003.2709
1709,0.91,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,6682,6807.2423


In [ ]:
unseen_predictions

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,Label
0,1.23,Very Good,G,VS1,EX,EX,GIA,8445,9072.6351
1,0.90,Fair,I,VS1,VG,EX,GIA,3526,3554.2894
2,0.77,Very Good,G,VVS1,EX,ID,AGSL,3966,4229.3503
3,1.51,Very Good,D,VS2,EX,EX,GIA,14416,14623.5531
4,2.33,Ideal,H,SI1,ID,ID,AGSL,21618,20527.9639
...,...,...,...,...,...,...,...,...,...
295,1.03,Ideal,D,SI1,EX,EX,GIA,6250,6742.9309
296,1.00,Very Good,D,SI1,VG,VG,GIA,5328,5621.3273
297,1.02,Ideal,D,SI1,EX,EX,GIA,6157,6679.0715
298,1.27,Signature-Ideal,G,VS1,EX,EX,GIA,11206,11642.7346


# 13.0 Deploy Model on Google Cloud

After the model is finalised and you are happy with the model, you can deploy the model on your cloud of choice. In this section, we deploy the model on the google cloud platform. 

In [21]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
! pip install awscli

In [22]:
# GCP project name, Change the name based on your own GCP project.
CLOUD_PROJECT = 'gcpessentials-rz' # GCP project name
bucket_name = 'pycaret-reg101-test1' # bucket name for storage of your model
BUCKET = 'gs://' + CLOUD_PROJECT + '-{}'.format(bucket_name)
# Set the gcloud consol to $CLOUD_PROJECT Environment Variable for your Desired Project)
!gcloud config set project $CLOUD_PROJECT

Updated property [core/project].


In [23]:
authentication = {'project': CLOUD_PROJECT, 'bucket' : bucket_name}
model_name = 'lightgbm-reg'
deploy_model(final_lightgbm, model_name, authentication, platform = 'gcp')

In [24]:
authentication = {'project': CLOUD_PROJECT, 'bucket' : bucket_name}
model_name = 'lightgbm-reg'
model_gcp = load_model(model_name, 
               platform = 'gcp', 
               authentication = authentication,
               verbose=True)

loading model from GCP
Transformation Pipeline and Model Successfully Loaded


In [25]:

unseen_predictions = predict_model(model_gcp, data=data_unseen, verbose=True)

In [26]:
unseen_predictions

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,Label
0,1.23,Very Good,G,VS1,EX,EX,GIA,8445,9072.6351
1,0.90,Fair,I,VS1,VG,EX,GIA,3526,3554.2894
2,0.77,Very Good,G,VVS1,EX,ID,AGSL,3966,4229.3503
3,1.51,Very Good,D,VS2,EX,EX,GIA,14416,14623.5531
4,2.33,Ideal,H,SI1,ID,ID,AGSL,21618,20527.9639
...,...,...,...,...,...,...,...,...,...
295,1.03,Ideal,D,SI1,EX,EX,GIA,6250,6742.9309
296,1.00,Very Good,D,SI1,VG,VG,GIA,5328,5621.3273
297,1.02,Ideal,D,SI1,EX,EX,GIA,6157,6679.0715
298,1.27,Signature-Ideal,G,VS1,EX,EX,GIA,11206,11642.7346
